# CO2 Emissions Dataset

## Requirements

- Virtual environment with Python 3.12.10
- pip install -r requirements.txt
- That includes:
    - jupyter
    - pandas
    - pyarrow
    - matplotlib
    - plotly

In [1]:
import pandas as pd

## Transforming to parquet

In [2]:
def csv_to_parquet(filename: str):
    pd.read_csv(filename).to_parquet(filename.removesuffix(".csv") + ".parquet")


csv_to_parquet("data/raw/Emissão_CO2_por_países.csv")

## Loading from parquet

In [3]:
df = pd.read_parquet("data/raw/Emissão_CO2_por_países.parquet")
df

,Country,Region,Date,Kilotons of Co2,Metric Tons Per Capita
0,Afghanistan,Asia,01-01-2011,8930.0,0.31
1,Afghanistan,Asia,01-01-2012,8080.0,0.27
2,None,Asia,01-01-2010,7110.0,0.25
3,None,Asia,01-01-2019,6080.0,0.16
4,None,Asia,01-01-2018,6070.0,0.17
...,...,...,...,...,...
5672,Zimbabwe,None,01-01-2004,9770.0,0.80
5673,Zimbabwe,None,01-01-2007,9760.0,0.78
5674,Zimbabwe,None,01-01-2010,9600.0,0.75
5675,Zimbabwe,None,01-01-2009,7750.0,0.61


# Data Cleaning

Renaming columns for convenience:

In [4]:
df = df.rename(
    columns={
        "Region": "region",
        "Country": "country",
        "Date": "date",
        "Kilotons of Co2": "kilotons",
        "Metric Tons Per Capita": "mtpc",
    }
)
df

,country,region,date,kilotons,mtpc
0,Afghanistan,Asia,01-01-2011,8930.0,0.31
1,Afghanistan,Asia,01-01-2012,8080.0,0.27
2,None,Asia,01-01-2010,7110.0,0.25
3,None,Asia,01-01-2019,6080.0,0.16
4,None,Asia,01-01-2018,6070.0,0.17
...,...,...,...,...,...
5672,Zimbabwe,None,01-01-2004,9770.0,0.80
5673,Zimbabwe,None,01-01-2007,9760.0,0.78
5674,Zimbabwe,None,01-01-2010,9600.0,0.75
5675,Zimbabwe,None,01-01-2009,7750.0,0.61


Searching for duplicated lines:

In [5]:
int(df.duplicated().sum()) # All lines are unique.

0

Getting informations about the dataset:

In [6]:
df.info()
# The date and mtpc columns have only non-null values.
# The other ones shall be treated for that.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5677 entries, 0 to 5676
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   country   5599 non-null   object 
 1   region    5653 non-null   object 
 2   date      5677 non-null   object 
 3   kilotons  5560 non-null   float64
 4   mtpc      5677 non-null   float64
dtypes: float64(2), object(3)
memory usage: 221.9+ KB


## Dealing with regions

Searching for typos:

In [7]:
df["region"].value_counts()

region
Africa      1558
Asia        1388
Europa      1221
Americas    1041
Oceania      408
Euro           9
Ocean          6
américa        5
ÁSia           5
ASIA           4
américas       3
frica          3
Ásia           1
améric         1
Name: count, dtype: int64

Correcting typos and standardizing the contents like the most frequent values:

In [8]:
df["region"] = df["region"].replace(
    {
        "Euro": "Europa",
        "Ocean": "Oceania",
        "américa": "Americas",
        "ÁSia": "Asia",
        "ASIA": "Asia",
        "américas": "Americas",
        "frica": "Africa",
        "Ásia": "Asia",
        "améric": "Americas",
    },
)
df["region"].value_counts()

region
Africa      1561
Asia        1398
Europa      1230
Americas    1050
Oceania      414
Name: count, dtype: int64

Visualizing nulls:

In [9]:
df[df["region"].isna()]

,country,region,date,kilotons,mtpc
5653,Zimbabwe,None,01-01-1995,15480.0,1.41
5654,Zimbabwe,None,01-01-1996,14860.0,1.33
5655,Zimbabwe,None,01-01-1998,14070.0,1.22
5656,Zimbabwe,None,01-01-2001,13900.0,1.17
5657,Zimbabwe,None,01-01-1997,13830.0,1.22
5658,Zimbabwe,None,01-01-2000,13700.0,1.16
5659,Zimbabwe,None,01-01-2002,12490.0,1.04
5660,Zimbabwe,None,01-01-2015,12430.0,0.88
5661,Zimbabwe,None,01-01-2018,12380.0,0.82
5662,Zimbabwe,None,01-01-2013,12280.0,0.91


Since Zimbabwe belongs to Africa, we may fill it in the dataframe directly:

In [10]:
df["region"] = df["region"].fillna("Africa")
df[df["region"].isna()]

,country,region,date,kilotons,mtpc


## Dealing with dates

All dates begin with "01-01":

In [11]:
int(df["date"].apply(lambda x: not str(x).startswith("01-01")).sum())

0

So it may be removed and the date column may be turned into an year column of integer type:

In [12]:
def remove_0101(obj):
    return obj.removeprefix("01-01-") if isinstance(obj, str) else obj


df["date"] = df["date"].apply(remove_0101).astype(int)
df = df.rename(columns={"date": "year"})
df

,country,region,year,kilotons,mtpc
0,Afghanistan,Asia,2011,8930.0,0.31
1,Afghanistan,Asia,2012,8080.0,0.27
2,None,Asia,2010,7110.0,0.25
3,None,Asia,2019,6080.0,0.16
4,None,Asia,2018,6070.0,0.17
...,...,...,...,...,...
5672,Zimbabwe,Africa,2004,9770.0,0.80
5673,Zimbabwe,Africa,2007,9760.0,0.78
5674,Zimbabwe,Africa,2010,9600.0,0.75
5675,Zimbabwe,Africa,2009,7750.0,0.61


Now checking values to search for typos:

In [13]:
date_values = df["year"].value_counts().to_frame()
date_values.T

year,2012,2019,2013,2018,2015,2016,2017,2014,2007,2006,...,24,21,9,200,2030,20100,0,1400,1427,1750
count,190,190,190,190,190,190,190,190,190,190,...,1,1,1,1,1,1,1,1,1,1


We can see that absurd values happen only one time:

In [14]:
wrong_dates = date_values.query("count == 1")
wrong_dates.T

year,2050,1600,2100,1500,1817,1000,201,205,24,21,9,200,2030,20100,0,1400,1427,1750
count,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


Since there is no way to discover the kind of mistake produced these values, the rows will be dropped:

In [15]:
df = df.drop(df[df["year"].apply(lambda x: x in wrong_dates.index)].index)
df

,country,region,year,kilotons,mtpc
0,Afghanistan,Asia,2011,8930.0,0.31
1,Afghanistan,Asia,2012,8080.0,0.27
2,None,Asia,2010,7110.0,0.25
3,None,Asia,2019,6080.0,0.16
4,None,Asia,2018,6070.0,0.17
...,...,...,...,...,...
5672,Zimbabwe,Africa,2004,9770.0,0.80
5673,Zimbabwe,Africa,2007,9760.0,0.78
5674,Zimbabwe,Africa,2010,9600.0,0.75
5675,Zimbabwe,Africa,2009,7750.0,0.61


Checking if absurd values are gone:

In [16]:
df["year"].value_counts().to_frame().T

year,2012,2019,2013,2018,2016,2015,2014,2017,2006,2007,...,2010,2005,2004,1999,1998,1996,2000,2001,1991,1990
count,190,190,190,190,190,190,190,190,190,190,...,188,188,188,188,188,188,188,187,185,183


Checking start year and final year of the series:

In [17]:
int(df["year"].min()), int(df["year"].max())

(1990, 2019)

## Dealing with countries

Visualizing nulls:

In [18]:
df[df["country"].isna()]

,country,region,year,kilotons,mtpc
2,None,Asia,2010,7110.0,0.25
3,None,Asia,2019,6080.0,0.16
4,None,Asia,2018,6070.0,0.17
5,None,Asia,2013,5990.0,0.19
6,None,Asia,2015,5950.0,0.18
...,...,...,...,...,...
1948,None,Americas,2009,810.0,7.15
1949,None,Americas,2012,410.0,3.54
1950,None,Americas,2019,330.0,2.69
1951,None,Americas,2018,320.0,2.63


Percentage of nulls:

In [19]:
f"{(df[df["country"].isna()].shape[0] / df.shape[0]):.2%}"

'1.38%'

These rows will be dropped, considering that:
1. the number of rows without country represents only 1.37% of the dataset; and
2. the lack of information about the country may indicate records with low reliability, since traceability is compromised.

In [20]:
df = df.dropna(subset="country")
df[df["country"].isna()]

,country,region,year,kilotons,mtpc


## Dealing with kilotons of Co2

Visualizing nulls:

In [21]:
df[df["kilotons"].isna()]

,country,region,year,kilotons,mtpc
107,Andorra,Europa,2013,NaN,6.73
108,Andorra,Europa,2017,NaN,6.37
109,Andorra,Europa,2016,NaN,6.48
110,Andorra,Europa,2015,NaN,6.55
111,Andorra,Europa,1997,NaN,7.27
...,...,...,...,...,...
1505,El Salvador,Americas,2008,NaN,1.14
1506,El Salvador,Americas,2005,NaN,1.13
1507,El Salvador,Americas,2018,NaN,1.09
1508,El Salvador,Americas,2011,NaN,1.10


Since we are dealing with multiple time series, we may backward-fill for the year-sorted subdataset of each country, and forward-fill when the last value is null.

In [22]:
df = df.sort_values(["country", "year"]).reset_index()
df

,index,country,region,year,kilotons,mtpc
0,12,Afghanistan,Asia,1990,2380.0,0.22
1,13,Afghanistan,Asia,1991,2230.0,0.21
2,17,Afghanistan,Asia,1992,1390.0,0.12
3,18,Afghanistan,Asia,1993,1340.0,0.10
4,19,Afghanistan,Asia,1994,1290.0,0.08
...,...,...,...,...,...,...
5576,5660,Zimbabwe,Africa,2015,12430.0,0.88
5577,5667,Zimbabwe,Africa,2016,11020.0,0.76
5578,5669,Zimbabwe,Africa,2017,10340.0,0.70
5579,5661,Zimbabwe,Africa,2018,12380.0,0.82


In [23]:
# For each country, if there is at least one null value of kilotons, then pandas
# must apply the backward-fill and forward-fill to ensure there will be no null
# values left.
for country in df["country"].unique().tolist():
    filter = df["country"] == country
    if df["kilotons"][filter].hasnans:
        df[filter] = df.query(f"country == '{country}'").bfill().ffill()
int(df["kilotons"].isna().sum())

0

## Saving clean dataset

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5581 entries, 0 to 5580
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     5581 non-null   int64  
 1   country   5581 non-null   object 
 2   region    5581 non-null   object 
 3   year      5581 non-null   int64  
 4   kilotons  5581 non-null   float64
 5   mtpc      5581 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 261.7+ KB


In [25]:
df.to_parquet("data/processed/co2_emissions.parquet")

# Data Analysis

In [26]:
df = pd.read_parquet("data/processed/co2_emissions.parquet")
df

,index,country,region,year,kilotons,mtpc
0,12,Afghanistan,Asia,1990,2380.0,0.22
1,13,Afghanistan,Asia,1991,2230.0,0.21
2,17,Afghanistan,Asia,1992,1390.0,0.12
3,18,Afghanistan,Asia,1993,1340.0,0.10
4,19,Afghanistan,Asia,1994,1290.0,0.08
...,...,...,...,...,...,...
5576,5660,Zimbabwe,Africa,2015,12430.0,0.88
5577,5667,Zimbabwe,Africa,2016,11020.0,0.76
5578,5669,Zimbabwe,Africa,2017,10340.0,0.70
5579,5661,Zimbabwe,Africa,2018,12380.0,0.82


## Total Emissions per Region

In this case we are interested in visualizing the number of kilotons of CO2 emitted per region.

In [27]:
import plotly.express as px

A first approach would be to compare the total per region for the whole period with data:

In [28]:
total_per_region = (
    df.groupby("region")["kilotons"].sum().to_frame().reset_index()
)
display(total_per_region)
px.bar(total_per_region, "region", "kilotons")

,region,kilotons
0,Africa,2.914535e+07
1,Americas,2.139168e+08
2,Asia,3.670914e+08
3,Europa,1.718793e+08
4,Oceania,1.004896e+07


## Total Emissions per Region by Year

However, a more interesting information would be the time evolution of emissions:

In [29]:
total_per_region_by_year = (
    df.groupby(["region", "year"])["kilotons"].sum().to_frame().reset_index()
)
display(total_per_region_by_year)
px.line(total_per_region_by_year, "year", "kilotons", color="region")

,region,year,kilotons
0,Africa,1990,635700.00
1,Africa,1991,650580.00
2,Africa,1992,658890.00
3,Africa,1993,671690.00
4,Africa,1994,682910.00
...,...,...,...
145,Oceania,2015,419419.99
146,Oceania,2016,427089.99
147,Oceania,2017,433150.00
148,Oceania,2018,430400.01


Now we see that Asia deeply increased its emissions, especially after 2000, while the rest of the world presented no trend.

## Mean Metric Tons per Capita per Region by Year

The problem with stopping here is that we would be comparing only absolute numbers, which may be misleading.

In fact, our analysis should address the change in population during this period, for example, and that could be done using the mean Metric Tons Per Capita (mtpc) per region by year:

In [30]:
mtpc_per_region_by_year = (
    df.groupby(["region", "year"])["mtpc"].mean().to_frame().reset_index()
)
display(mtpc_per_region_by_year)
px.line(mtpc_per_region_by_year, "year", "mtpc", color="region")

,region,year,mtpc
0,Africa,1990,0.796400
1,Africa,1991,0.792885
2,Africa,1992,0.813654
3,Africa,1993,0.817736
4,Africa,1994,0.843585
...,...,...,...
145,Oceania,2015,3.674286
146,Oceania,2016,3.942857
147,Oceania,2017,4.092857
148,Oceania,2018,3.925714


Now it is clear from context that the mean metric tons per capita emitted in Asia has not increased much, which means that a fast population growth with approximately the same level of efficiency in terms of energy generation has lead to the sharp rise in emissions.